In [1]:
import pandas as pd
import pvlib

FILE_IN = r'.data/psm3_demo.csv'
FILE_OUT = r'.data/psm3_pytest_2.csv'

In [2]:
header = []
with open(FILE_IN, 'r') as file:
    header.append(file.readline())
    header.append(file.readline())

data, meta = pvlib.iotools.read_psm3(FILE_IN)

c:\users\mbrown2\documents\github\pvlib\pvlib\iotools\psm3.py:349: pvlibDeprecationWarning: PSM3 variable names will be renamed to pvlib conventions by default starting in pvlib 0.11.0. Specify map_variables=True to enable that behavior now, or specify map_variables=False to hide this warning.
  warnings.warn(


In [3]:
dni = data.DNI
dhi = data.DHI
ghi = data.GHI
time_range = data.index

solar_pos = pvlib.solarposition.get_solarposition(time=time_range, latitude=meta['Latitude'],
                                            longitude=meta['Longitude'], altitude=meta['Elevation'])

poa_df = pvlib.irradiance.get_total_irradiance(surface_tilt=20, surface_azimuth=180, 
                                                    solar_zenith=solar_pos.apparent_zenith,
                                                    solar_azimuth=solar_pos.azimuth,
                                                    dni=dni, ghi=ghi, dhi=dhi)
poa_global = poa_df.poa_global

In [4]:
cell_params = pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_polymer']

temp_cell = pvlib.temperature.sapm_cell(poa_global=poa_global, temp_air=data['Temperature'],
                                        wind_speed=data['Wind Speed'], **cell_params)

In [5]:
temp_module = pvlib.temperature.sapm_module(poa_global=poa_global, temp_air=data['Temperature'],
                                            wind_speed=data['Wind Speed'], a=cell_params['a'],
                                            b=cell_params['b'])

In [6]:
data['poa_global'] = poa_global
data['temp_cell'] = temp_cell
data['temp_module'] = temp_module
data['time_range'] = time_range

In [7]:
with open(FILE_OUT, 'w') as file:
    for line in header:
        file.write(line)
    data.to_csv(file, index=False)